In [48]:
import requests
import json
import sqlite3
import logging

## Definition function

In [33]:
def get_buienradar_data():
    url = "https://json.buienradar.nl/"
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # raises exception for HTTP errors

        data = response.json()  # convert JSON to Python dict
        return data

    except requests.exceptions.RequestException as e:
        print(f"Error fetching Buienradar data: {e}")
        return None


In [34]:
def get_connection(db_name="buienradar_weather.db"):
    conn = sqlite3.connect(db_name)
    conn.execute("PRAGMA foreign_keys = ON;")
    return conn

In [44]:
# Step 1: Create database schema
def setup_database(conn):
    cursor = conn.cursor()

    # Station table — use stationid from Buienradar as PRIMARY KEY (no AUTOINCREMENT)
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS weather_stations (
            stationid INTEGER PRIMARY KEY,
            stationname TEXT,
            lat REAL,
            lon REAL,
            regio TEXT
        )
    """)

    # Measurements table — link stationid as foreign key
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS weather_station_measurements (
            measurementid INTEGER PRIMARY KEY AUTOINCREMENT,
            timestamp DATETIME,
            temperature REAL,
            groundtemperature REAL,
            feeltemperature REAL,
            windgusts REAL,
            windspeedBft REAL,
            humidity REAL,
            precipitation REAL,
            sunpower REAL,
            stationid INTEGER,
            FOREIGN KEY (stationid) REFERENCES weather_stations(stationid)
                ON UPDATE CASCADE
                ON DELETE CASCADE
        )
    """)
    conn.commit()

## Testing Zone

In [32]:
url = "https://json.buienradar.nl/"
response= requests.get(url, timeout=10)
data = response.json()

stations = data.get("actual", {}).get("stationmeasurements", [])
stations[0]


{'$id': '4',
 'stationid': 6391,
 'stationname': 'Meetstation Arcen',
 'lat': 51.5,
 'lon': 6.2,
 'regio': 'Venlo',
 'timestamp': '2025-10-22T14:00:00',
 'weatherdescription': 'Zwaar bewolkt',
 'iconurl': 'https://cdn.buienradar.nl/resources/images/icons/weather/30x30/c.png',
 'fullIconUrl': 'https://cdn.buienradar.nl/resources/images/icons/weather/96x96/C.png',
 'graphUrl': 'https://www.buienradar.nl/nederland/weerbericht/weergrafieken/c',
 'winddirection': 'WZW',
 'temperature': 14.9,
 'groundtemperature': 15.0,
 'feeltemperature': 14.9,
 'windgusts': 3.7,
 'windspeed': 1.8,
 'windspeedBft': 2,
 'humidity': 77.0,
 'precipitation': 0.0,
 'sunpower': 239.0,
 'rainFallLast24Hour': 0.2,
 'rainFallLastHour': 0.0,
 'winddirectiondegrees': 243}

In [38]:
# Step 2: Insert or update stations
def insert_stations(conn, data):
    cursor = conn.cursor()
    stations = data.get("actual", {}).get("stationmeasurements", [])

    for s in stations:
        cursor.execute("""
            INSERT INTO weather_stations (stationid, stationname, lat, lon, regio)
            VALUES (?, ?, ?, ?, ?)
            ON CONFLICT(stationid) DO UPDATE SET
                stationname=excluded.stationname,
                lat=excluded.lat,
                lon=excluded.lon,
                regio=excluded.regio;
        """, (
            s.get("stationid"),
            s.get("stationname"),
            s.get("lat"),
            s.get("lon"),
            s.get("regio")
        ))

    conn.commit()
    print(f"✅ Inserted/updated {len(stations)} stations.")

In [45]:
conn = get_connection()
setup_database(conn)


In [39]:
insert_stations(conn, data)

✅ Inserted/updated 40 stations.


In [46]:
# Step 3: Insert measurements linked by stationid
def insert_measurements(conn, data):
    cursor = conn.cursor()
    stations = data.get("actual", {}).get("stationmeasurements", [])

    for s in stations:
        cursor.execute("""
            INSERT INTO weather_station_measurements
            (timestamp, temperature, groundtemperature, feeltemperature, windgusts, windspeedBft,
             humidity, precipitation, sunpower, stationid)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            s.get("timestamp"),
            s.get("temperature"),  
            s.get("groundtemperature"),
            s.get("feeltemperature"),
            s.get("windgusts"),
            s.get("windspeedBft"),
            s.get("humidity"),
            s.get("precipitation"),
            s.get("sunpower"),
            s.get("stationid")
        ))

    conn.commit()
    print(f"✅ Inserted {len(stations)} station measurements.")

In [47]:
insert_measurements(conn, data)

✅ Inserted 40 station measurements.
